In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
### Latex font in plots
plt.rcParams['font.serif'] = "cmr10"
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24

plt.rcParams['legend.fontsize'] = 20
plt.rcParams['legend.loc'] = 'upper right'
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams["axes.grid"] = True
# plt.rcParams['text.usetex'] = True
plt.rc('axes', unicode_minus=False)


In [ ]:
# Load the CSV file to examine its structure
file_path = '../data/k-barrier-tuning/test-data/first-half-year-experiment.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()

In [ ]:

# Convert 'Time' to readable datetime format
data['Date'] = pd.to_datetime(data['Time'], unit='ms')
data['Date'] = data['Date'].apply(lambda t: t+timedelta(hours=1))


In [ ]:

# Extract time of day from the 'Date' column
data['TimeOfDay'] = data['Date'].dt.time

# Group by time of day and calculate the mean temperature for each time step across all runs
time_step_means = data.groupby('TimeOfDay')['Air Temperature'].mean().reset_index()

# Convert 'TimeOfDay' to minutes past midnight for plotting
time_step_means['MinutesPastMidnight'] = time_step_means['TimeOfDay'].apply(lambda t: t.hour + t.minute/60)

# Plotting with the modified time format
plt.figure(figsize=(10, 6))
plt.plot(time_step_means['MinutesPastMidnight'], time_step_means['Air Temperature'], marker='o', linestyle='-', color=(0.9, .3, 0))
plt.title('Average Temperature per Time Step Across All Runs')
plt.xlabel('Hour of the day')
plt.ylabel(r'Average Temperature [$^\circ$C]')
plt.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
# Group by time of day and calculate the mean CO2 resource usage for each time step across all runs
co2_resource_means = data.groupby('TimeOfDay')['CO2 resource'].mean().reset_index()

# Adjust 'TimeOfDay' by adding one hour for UTC+1
# co2_resource_means['TimeOfDayUTC+1'] = co2_resource_means['TimeOfDay'].apply(lambda t: (datetime.combine(datetime.today(), t) + timedelta(hours=1)).time())

# Convert 'TimeOfDayUTC+1' to hours past midnight for plotting
co2_resource_means['HoursPastMidnightUTC+1'] = co2_resource_means['TimeOfDay'].apply(lambda t: t.hour + t.minute / 60)

# Plotting the CO2 resource usage against hours past midnight (UTC+1)
plt.figure(figsize=(10, 6))
plt.plot(co2_resource_means['HoursPastMidnightUTC+1'], co2_resource_means['CO2 resource'], marker='o', markersize=3, linestyle='-', color=(0.9, .3, 0))
# plt.title('Average CO2 Resource Usage per Time Step Across All Runs (UTC+1)')

plt.vlines(18, ymin=0, ymax=0.002, linestyle="--", color='gray')
plt.vlines(2, ymin=0, ymax=0.002, linestyle="--", color='gray')
plt.ylim(0, 0.002)
# Filling the area between 02:00 and 18:00 with a lighter gray color
plt.fill_betweenx([plt.ylim()[0], plt.ylim()[1]], 2, 18, color='gray', alpha=0.2)

middle_point = (2 + 18) / 2
plt.text(middle_point, plt.ylim()[1]*0.9, 'Lamps on', horizontalalignment='center', verticalalignment='center', fontsize=18, color='black', alpha=0.7)
plt.xlabel('Hour of the day')
plt.ylabel(r'CO$_2$ injection (kg m$^{-2}$)')

plt.grid(True)
plt.tight_layout()
plt.savefig('co2_injection.svg')
plt.show()

In [ ]:
# Group by time of day and calculate the mean CO2 resource usage for each time step across all runs
co2_resource_means = data.groupby('TimeOfDay')['CO2 concentration'].mean().reset_index()

# Adjust 'TimeOfDay' by adding one hour for UTC+1
# co2_resource_means['TimeOfDayUTC+1'] = co2_resource_means['TimeOfDay'].apply(lambda t: (datetime.combine(datetime.today(), t) + timedelta(hours=1)).time())

# Convert 'TimeOfDayUTC+1' to hours past midnight for plotting
co2_resource_means['HoursPastMidnightUTC+1'] = co2_resource_means['TimeOfDay'].apply(lambda t: t.hour + t.minute / 60)

# Plotting the CO2 resource usage against hours past midnight (UTC+1)
plt.figure(figsize=(10, 6))
plt.plot(co2_resource_means['HoursPastMidnightUTC+1'], co2_resource_means['CO2 concentration'], marker='o', markersize=3, linestyle='-', color=(0.9, .3, 0))
# plt.title('Average CO2 Resource Usage per Time Step Across All Runs (UTC+1)')

plt.vlines(18, ymin=0, ymax=0.002, linestyle="--", color='gray')
plt.vlines(2, ymin=0, ymax=0.002, linestyle="--", color='gray')
# plt.ylim(0, 0.002)
# Filling the area between 02:00 and 18:00 with a lighter gray color
plt.fill_betweenx([plt.ylim()[0], plt.ylim()[1]], 2, 18, color='gray', alpha=0.2)

middle_point = (2 + 18) / 2
plt.text(middle_point, plt.ylim()[1]*0.9, 'Lamps on', horizontalalignment='center', verticalalignment='center', fontsize=18, color='black', alpha=0.7)
plt.xlabel('Hour of the day')
plt.ylabel(r'CO$_2$ injection [kg m$^{-2}$]')

# plt.grid(True)
plt.tight_layout()
plt.savefig('co2_injection.svg')
plt.show()